<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_time_series/class_01/TS%20-%20W1%20-%2003a%20-%20Introduction%20to%20time%20series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Introdução aos gráficos de séries temporais

Este notebook tem como objetivo mostrar diferentes terminologias e atributos de uma série temporal gerando e plotando dados sintéticos.

Experimentar diferentes modelos de previsão nesse tipo de dados é uma boa maneira de desenvolver sua intuição quando você começar a trabalhar com dados do mundo real mais adiante no curso. Vamos começar!

## Importações

Você usará principalmente as bibliotecas [Numpy](https://numpy.org) e [Matplotlib's Pyplot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html) para gerar os dados e plotar os gráficos.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Utilitários de plotagem

Você plotará vários gráficos neste notebook, portanto, é bom ter uma função utilitária para isso.

O código a seguir permite visualizar matrizes numpy em um gráfico usando o método [plot()](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html) do Pyplot.

O eixo x conterá as etapas de tempo. A unidade exata não é essencial para este exercício, portanto, você pode imaginar que são segundos, horas, ano etc. O eixo y conterá os valores medidos em cada etapa de tempo.

In [ ]:
def plot_series(time, series, format="-", start=0, end=None, label=None):
    """
    Visualiza dados de séries temporais

    Args:
      time (vetor de int) - contém as etapas de tempo
      series (vetor de int) - contém as medições de cada etapa de tempo
      format (string) - estilo de linha ao plotar o gráfico
      start (int) - primeiro passo de tempo a ser plotado
      end (int) - último passo de tempo a ser plotado
      label (lista de strings) - etiqueta para a linha
    """

    # Defina as dimensões da figura do gráfico
    plt.figure(figsize=(10, 6))

    # Plotar os dados da série temporal
    plt.plot(time[start:end], series[start:end], format)

    # Rotular o eixo x
    plt.xlabel("Tempo")

    # Rotular o eixo y
    plt.ylabel("Valor")

    if label:
      plt.legend(fontsize=14, labels=label)

    # Sobrepor uma grade no gráfico
    plt.grid(True)

    # Desenhe o gráfico na tela
    plt.show()

## Tendência

A *tendência* descreve a tendência geral dos valores em subir ou descer com o passar do tempo.

Dado um determinado período de tempo, você pode ver se o gráfico está seguindo uma tendência ascendente/positiva, descendente/negativa ou simplesmente estável.

Por exemplo, os preços de imóveis em uma boa localização podem apresentar um aumento geral na avaliação com o passar do tempo.

O exemplo mais simples de visualizar são os dados que seguem uma linha reta. Você usará a função abaixo para gerar isso. O argumento `slope` determinará qual é a tendência. É uma [forma de interceptação de inclinação](https://en.wikipedia.org/wiki/Linear_equation#Slope%E2%80%93intercept_form_or_Gradient-intercept_form) com a interceptação y sendo `0`.

In [ ]:
def trend(time, slope=0):
    """
    Gera dados sintéticos que seguem uma linha reta com um valor de inclinação.

    Args:
      time (vetor de int) - contém as etapas de tempo
      slope (float) - determina a direção e a inclinação da linha

    Retorna:
      series (array of float) - medições que seguem uma linha reta
    """

    # Calcula a série linear dada a inclinação
    series = slope * time

    return series

Aqui está uma série temporal com tendência de alta.

Para uma tendência de queda, basta substituir o valor da inclinação abaixo por um valor negativo (por exemplo, `-0,3`).

In [ ]:
# Gerar etapas de tempo. Suponha que seja 1 por dia durante um ano (365 dias)
time = np.arange(365)

# Defina a inclinação (Você pode revisar isso)
slope = 0.1

# Gerar medições com a inclinação definida
series = trend(time, slope)

# Plotar os resultados
plot_series(time, series, label=[f'inclinação={slope}'])

Como você pode ver, não é necessário o aprendizado de máquina para modelar esse comportamento.

Você pode simplesmente resolver a equação da linha e terá o modelo de previsão perfeito.

No entanto, dados como esses são extremamente raros em aplicações do mundo real e a linha de tendência é usada simplesmente como um guia, como no exemplo da [Lei de Moore](https://en.wikipedia.org/wiki/Moore%27s_law).

## Sazonalidade

Outro atributo que você pode querer procurar é a sazonalidade.

Isso se refere a um padrão recorrente em intervalos regulares de tempo.
> Por exemplo, a temperatura horária pode oscilar de forma semelhante por 10 dias consecutivos e você pode usar isso para prever o comportamento no dia seguinte.

Você pode usar as funções abaixo para gerar uma série temporal com um padrão sazonal:

In [ ]:
def seasonal_pattern(season_time):
    """
    Apenas um padrão arbitrário, você pode alterá-lo se desejar

    Args:
      season_time (vetor de float) - contém as medições por etapa de tempo

    Retorna:
      data_pattern (vetor de float) - contém os valores de medição revisados de acordo com o padrão definido
    """
    # Gerar os valores usando um padrão arbitrário
    data_pattern = np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

    return data_pattern

def seasonality(time, period, amplitude=1, phase=0):
    """
    Repete o mesmo padrão em cada período

    Args:
      time (vetor de int) - contém as etapas de tempo
      period (int) - número de etapas de tempo antes da repetição do padrão
      amplitude (int) - valor de pico medido em um período
      phase (int) - número de etapas de tempo para deslocar os valores medidos

    Retorna:
      data_pattern (vetor de float) - dados sazonais dimensionados pela amplitude definida
    """
    # Definir os valores medidos por período
    season_time = ((time + phase) % period) / period

    # Gera os dados sazonais dimensionados pela amplitude definida
    data_pattern = amplitude * seasonal_pattern(season_time)

    return data_pattern

A célula abaixo mostra a sazonalidade dos dados gerados porque você pode ver o padrão a cada 365 etapas de tempo.

In [ ]:
# Gerar etapas de tempo
time = np.arange(4 * 365 + 1)

# Definir os parâmetros dos dados sazonais
period = 365
amplitude = 40

# Gerar os dados sazonais
series = seasonality(time, period=period, amplitude=amplitude)

# Plotar os resultados
plot_series(time, series)

Uma série temporal também pode conter tendência e sazonalidade. Por exemplo, a temperatura horária pode oscilar regularmente em períodos curtos, mas pode mostrar uma tendência de alta se você analisar os dados de vários anos.

O exemplo abaixo demonstra um padrão sazonal com uma tendência de alta:

In [ ]:
# Definir parâmetros sazonais
slope = 0.05
period = 365
amplitude = 40

# Gerar os dados
series = trend(time, slope) + seasonality(time, period=period, amplitude=amplitude)

# Plotar os resultados
plot_series(time, series)

## Ruído

Na prática, poucas séries temporais da vida real têm um sinal tão suave.

Elas geralmente têm algum ruído sobre esse sinal.

As próximas células mostrarão a aparência de um sinal com ruído:

In [ ]:
def noise(time, noise_level=1, seed=None):
    """Gera um sinal ruidoso normalmente distribuído

    Args:
      time (vetor de int) - contém as etapas de tempo
      noise_level (float) - fator de escala para o sinal gerado
      seed (int) - semente do gerador de números para repetibilidade

    Retorna:
      noise (vetor de float) - o sinal ruidoso

    """

    # Inicializar o gerador de números aleatórios
    rnd = np.random.RandomState(seed)

    # Gerar um número aleatório para cada etapa de tempo e dimensionar pelo nível de ruído
    noise = rnd.randn(len(time)) * noise_level

    return noise

In [ ]:
# Definir o nível de ruído
noise_level = 5

# Gerar sinal com ruído
noise_signal = noise(time, noise_level=noise_level, seed=42)

# Plotar os resultados
plot_series(time, noise_signal)

Agora vamos adicionar isso à série temporal que geramos anteriormente:

In [ ]:
# Adicione o ruído à série temporal
series += noise_signal

# Plotar os resultados
plot_series(time, series)

Como você pode ver, a série ainda está tendendo para cima e é sazonal, mas há mais variação entre as etapas de tempo devido ao ruído adicionado.

## Autocorrelação

As séries temporais também podem ser autocorrelacionadas.

Isso significa que as medições em uma determinada etapa de tempo são uma função das etapas de tempo anteriores. Serão apresentadas algumas funções que demonstram isso.

Observe as linhas que se referem à variável `step` porque é nela que ocorre o cálculo das etapas de tempo anteriores.

Também será incluído ruído (ou seja, números aleatórios) para tornar o resultado um pouco mais realista.

In [ ]:
def autocorrelation(time, amplitude, seed=None):
    """
    Gera dados autocorrelacionados

    Args:
      time (vetor de int) - contém as etapas de tempo
      amplitude (float) - fator de escala
      seed (int) - semente do gerador de números para repetibilidade

    Retorna:
      ar (vetor de float) - dados autocorrelacionados
    """

    # Inicializar o gerador de números aleatórios
    rnd = np.random.RandomState(seed)

    # Inicialize o vetor de números aleatórios igual ao comprimento
    # das etapas de tempo fornecidas mais 50
    ar = rnd.randn(len(time) + 50)

    # Definir os primeiros 50 elementos como uma constante
    ar[:50] = 100

    # Definir fatores de escala
    phi1 = 0.5
    phi2 = -0.1

    # Autocorrelacione o elemento 51 em diante com a medição em
    # (t-50) e (t-30), em que t é o intervalo de tempo atual
    for step in range(50, len(time) + 50):
        ar[step] += phi1 * ar[step - 50]
        ar[step] += phi2 * ar[step - 33]

    # Obtenha os dados autocorrelacionados e dimensione-os com a amplitude fornecida.
    # Os primeiros 50 elementos da matriz original são truncados porque
    # esses são apenas constantes e não autocorrelacionados.
    ar = ar[50:] * amplitude

    return ar

In [ ]:
# Use as etapas de tempo da seção anterior e gere dados autocorrelacionados
series = autocorrelation(time, amplitude=10, seed=42)

# Plote os primeiros 200 elementos para ver o padrão com mais clareza
plot_series(time[:200], series[:200])

Aqui está uma função de autocorrelação mais direta que calcula apenas um valor da etapa de tempo anterior.

In [ ]:
def autocorrelation(time, amplitude, seed=None):
    """
    Gera dados autocorrelacionados

    Args:
      time (vetor de int) - contém as etapas de tempo
      amplitude (float) - fator de escala
      seed (int) - semente do gerador de números para repetibilidade

    Retorna:
      ar (vetor de float) - dados autocorrelacionados gerados
    """

    # Inicializar o gerador de números aleatórios
    rnd = np.random.RandomState(seed)

    # Inicialize o vetor  de números aleatórios igual ao comprimento
    # das etapas de tempo fornecidas mais uma etapa adicional
    ar = rnd.randn(len(time) + 1)

    # Definir fator de escala
    phi = 0.8

    # Autocorrelacione o elemento 11 em diante com a medição em
    # (t-1), em que t é o intervalo de tempo atual
    for step in range(1, len(time) + 1):
        ar[step] += phi * ar[step - 1]

    # Obtenha os dados autocorrelacionados e dimensione-os com a amplitude fornecida.
    ar = ar[1:] * amplitude

    return ar

In [ ]:
# Use as etapas de tempo da seção anterior e gere dados autocorrelacionados
series = autocorrelation(time, amplitude=10, seed=42)

# Plotar os resultados
plot_series(time[:200], series[:200])

Outra série temporal autocorrelacionada que você pode encontrar é aquela em que ela decai de forma previsível após picos aleatórios.

Você definirá primeiro a função que gera esses picos abaixo.

In [ ]:
def impulses(time, num_impulses, amplitude=1, seed=None):
    """
    Gera impulsos aleatórios

    Args:
      time (vetor de int) - contém as etapas de tempo
      num_impulses (int) - número de impulsos a serem gerados
      amplitude (float) - fator de escala
      seed (int) - semente do gerador de números para repetibilidade

    Retorna:
      series (vetor de float) - array que contém os impulsos
    """

    # Inicializar o gerador de números aleatórios
    rnd = np.random.RandomState(seed)

    # Gerando números randômicos
    impulse_indices = rnd.randint(len(time), size=num_impulses)

    # Inicializar series
    series = np.zeros(len(time))

    # Inserir impulsos randômicos
    for index in impulse_indices:
        series[index] += rnd.rand() * amplitude

    return series

Você usará a função acima para gerar uma série com 10 impulsos aleatórios.

In [ ]:
# Gerar impulsos aleatórios
impulses_signal = impulses(time, num_impulses=10, seed=42)

# Plotar os resultados
plot_series(time, impulses_signal)

Agora que você tem a série, você definirá a função que decairá os próximos valores após o pico.

In [ ]:
def autocorrelation_impulses(source, phis):
    """
    Gera dados autocorrelacionados a partir de impulsos

    Args:
      source (vetor de float) - contém as etapas de tempo com impulsos
      phis (dict) - dicionário que contém o tempo de defasagem e as taxas de decaimento

    Retorna:
      ar (vetor de float) - dados autocorrelacionados gerados
    """

    # Copiar a fonte
    ar = source.copy()

    # Calcular novos valores de série com base nos tempos de defasagem e nas taxas de decaimento
    for step, value in enumerate(source):
        for lag, phi in phis.items():
            if step - lag > 0:
              ar[step] += phi * ar[step - lag]

    return ar

Em seguida, você pode usar a função para gerar o decaimento após os picos.

Aqui está um exemplo que gera o próximo valor a partir da etapa de tempo anterior (ou seja, `t-1`, em que `t` é a etapa de tempo atual):

In [ ]:
# Use os impulsos da seção anterior e gere dados autocorrelacionados
series = autocorrelation_impulses(impulses_signal, {1: 0.99})

# Plotar os resultados
plot_series(time, series)

Aqui está outro exemplo em que os próximos valores são calculados a partir daqueles em `t-1` e `t-50`:

In [ ]:
# Use os impulsos da seção anterior e gere dados autocorrelacionados
series = autocorrelation_impulses(impulses_signal, {1: 0.70, 50: 0.2})

# Plotar os resultados
plot_series(time, series)

Os dados autocorrelacionados também podem seguir uma linha de tendência, que terá a aparência abaixo.

In [ ]:
# Gerar dados autocorrelacionados com uma tendência ascendente
series = autocorrelation(time, 10, seed=42) + trend(time, 2)

# Plotar os resultados
plot_series(time[:200], series[:200])

Da mesma forma, a sazonalidade também pode ser adicionada a esses dados.

In [ ]:
# Gerar dados autocorrelacionados com uma tendência ascendente
series = autocorrelation(time, 10, seed=42) + seasonality(time, period=50, amplitude=150) + trend(time, 2)

# Plotar os resultados
plot_series(time[:200], series[:200])


## Séries temporais não estacionárias

Também é possível que a série temporal quebre um padrão esperado.

Grandes eventos podem alterar a tendência ou o comportamento sazonal dos dados.

Seria algo parecido com o que está abaixo, em que o gráfico mudou para uma tendência de queda na etapa de tempo = 200.

In [ ]:
# Gerar dados com tendência positiva
series = autocorrelation(time, 10, seed=42) + seasonality(time, period=50, amplitude=150) + trend(time, 2)

# Gerar dados com tendência negativa
series2 = autocorrelation(time, 5, seed=42) + seasonality(time, period=50, amplitude=2) + trend(time, -1) + 550

# Divida os dados de tendência decrescente no primeiro, na etapa de tempo = 200
series[200:] = series2[200:]

# Plote o resultado
plot_series(time[:300], series[:300])

Em casos como esse, talvez você queira treinar o modelo nas etapas posteriores (ou seja, a partir de t=200), pois elas apresentam um sinal de previsão mais forte para as etapas futuras.

## Resumo

Isso conclui esta introdução às terminologias e atributos de séries temporais. Você também viu como gerá-los e os usará para testar diferentes técnicas de previsão nas próximas seções. Vejo você lá!